### Входные данные
Форма для сдачи: https://goo.gl/forms/qptFQ1ukuCxeMEA63

У вас имеется поток данных (генератор data_stream). Поля это случайные величины - так сделано для упрощения генерации данных. Есть три поля (названы по уровню сложности задания)

### Задание
#####Мотивация:
У вас есть куча временных рядов, вы хотите научиться предсказывать следующее значение по 1000 предыдущим. 1000 признаков окна это слишком много, однако вы решили заменить их 5ю: средним, дисперсией, минимумом, медианой и максимумом. Однако, все эти признаки надо подсчитать, причём хочется уметь это делать быстро (в течение часа)
#####Для каждого поля нужно сделать следующее:

1. Пробежаться по данным окном размера 1000 (окно сдвигается на 1, то есть следующее окно пересекается с предыдущим по 999 элементам).

2. Для каждого окна посчитайте среднее значение поля и его дисперсию. Делайте yield этих значений, получая генератор tuple. 

3. Для каждого окна найдине минимум, медиану и максимум в нём. Делайте yield этих значений, получая генератор tuple. 

Ответом, который нужно будет засабмитить в гугл форму, является среднее значение tuple по получившемуся потоку, округлённое до 3 знака после запятой.

### Замечания

1. Обратите внимания как генерируются поля. Постарайтесь понять особенность каждого поля и как это можно использовать. Желательно, чтобы для каждого поля у вас было своё решение, максимально эффективно использующее знание об этом поле.
2. Полезные библиотеки: itertools, numpy, collections + всё что найдёте в интернете и можно поставить через pip install



Если измерять время работы функций временем работы функции example, то примерное время работы такое:
Одновременно среднее, дисперсия - 1.17
Одновременно минимум, максимум и медиана:easy - 0.87
medium - 2.11
nightmare - 2.85


#### Генерация данных

In [143]:
from collections import namedtuple
from collections import deque
import random
import itertools 
import heapq
import numpy as np
import tqdm

Record = namedtuple('Record', 'easy medium nightmare')

def data_stream():
    random_generator = random.Random(42)
    easy = 0
    for _ in xrange(10000000):
        easy += random_generator.randint(0, 2) 
        medium = random_generator.randint(0, 256 - 1)
        nightmare = random_generator.randint(0, 1000000000 - 1)

        yield Record(
            easy=easy,
            medium=medium,
            nightmare=nightmare
        )
        
def easy_stream():
    for record in data_stream():
        yield record.easy
        
def medium_stream():
    for record in data_stream():
        yield record.medium
        
def nightmare_stream():
    for record in data_stream():
        yield record.nightmare

#### Подсчёт среднего значения tuple по потоку

In [59]:
import numpy as np

def get_tuple_stream_mean(stream, number_of_values):
    result = np.zeros(number_of_values, dtype='object')
    count = 0. 
    for streamed_tuple in stream:
        result += streamed_tuple
        count += 1
    return [round(x, 3) for x in result / count]

In [140]:
def example(stream):
    for value in stream:
        yield (value, value + 10)
get_tuple_stream_mean(example(easy_stream()), 2)

KeyboardInterrupt: 

In [6]:
mean_var_tuple = namedtuple('mean_var_tuple', 'mean var')

def mean_var_stream(stream, window_len):
    values_array = np.array([stream.next() for i in range(window_len)])
    mean = np.mean(values_array)
    var = np.var(values_array)
    
    queue = deque(maxlen=window_len)
    for i in range(values_array.size):
        queue.append(values_array[i])
    values_array = None
    yield mean_var_tuple(mean = mean, var = var)
    
    for value_in in stream:
        value_in = float(value_in)
        value_out = float(queue.popleft())
        queue.append(value_in)
        delta_mean = (value_in - value_out)/window_len
        var += (value_in**2 - value_out**2)/window_len - 2*mean*delta_mean - delta_mean**2
        mean += delta_mean
        yield mean_var_tuple(mean = mean, var = var)

In [7]:
array = np.random.randint(0, 1000000000 - 1, 100)
def test_stream(array):
    for i in array:
        yield i

window_len = 10
stream = mean_var_stream(test_stream(array), window_len)
i = 0
while i + window_len <= array.size:
    #print array[i:i+window_len]
    next = stream.next()
    #print  np.var(array[i:i+window_len]), next[1] 
    print np.mean(array[i:i+window_len]) - next[0], np.var(array[i:i+window_len]) - next[1]
    i += 1

0.0 0.0
0.0 -16.0
0.0 -16.0
0.0 -32.0
0.0 -8.0
0.0 0.0
0.0 -16.0
0.0 -16.0
0.0 -16.0
0.0 -16.0
0.0 -16.0
0.0 -32.0
0.0 -32.0
5.96046447754e-08 -32.0
5.96046447754e-08 -32.0
1.19209289551e-07 -16.0
5.96046447754e-08 -16.0
0.0 0.0
0.0 -16.0
0.0 0.0
5.96046447754e-08 16.0
0.0 -32.0
0.0 -32.0
0.0 -16.0
0.0 -16.0
0.0 -32.0
0.0 -24.0
5.96046447754e-08 -32.0
5.96046447754e-08 -32.0
5.96046447754e-08 -16.0
5.96046447754e-08 0.0
5.96046447754e-08 -32.0
5.96046447754e-08 -16.0
0.0 -32.0
0.0 -48.0
0.0 -32.0
0.0 -32.0
0.0 -32.0
-1.19209289551e-07 -32.0
0.0 -48.0
0.0 -48.0
-1.19209289551e-07 -48.0
-1.19209289551e-07 -64.0
-1.19209289551e-07 -80.0
-1.19209289551e-07 -64.0
-1.19209289551e-07 -32.0
-1.19209289551e-07 -48.0
-2.38418579102e-07 -64.0
-1.78813934326e-07 -96.0
-1.78813934326e-07 -96.0
-1.78813934326e-07 -96.0
-1.19209289551e-07 -96.0
-1.78813934326e-07 -112.0
-1.78813934326e-07 -128.0
-1.78813934326e-07 -128.0
-1.19209289551e-07 -160.0
-1.78813934326e-07 -168.0
-1.19209289551e-07 -156.0
-1

In [ ]:
(array[-8:]**2).mean() - array[-8:].mean()**2

In [44]:
from sortedcontainers import SortedList
min_median_max_tuple = namedtuple('min_median_max_tuple', 'min median max')

def min_median_max_stream(stream, window_len):
    values = np.array([stream.next() for i in range(window_len)])
    Min, Max = np.min(values), np.max(values)
    median = np.median(values)
    
    queue = deque(maxlen = window_len)
    for i in range(values.size):
        queue.append(values[i])
        
    values = SortedList(values)
    yield min_median_max_tuple(min = Min, median = median, max = Max)
    
    for value_in in stream:
        value_out = queue.popleft()
        values.remove(value_out)
        queue.append(value_in)
        values.add(value_in)
        median = (values[int(np.floor((window_len-1)/2.0))] + values[int(np.ceil((window_len-1)/2.0))])/2.0
        Min = values[0]
        Max = values[-1]
        yield min_median_max_tuple(min = Min, median = median, max = Max)

In [147]:
array = np.random.randint(0, 1000000000 - 1, 1000)
#array = np.array(range(1000))
def test_stream(array):
    for i in array:
        yield i

window_len = 100
stream = min_median_max_stream(test_stream(array), window_len)
i = 0
while i + window_len <= array.size:
    next = stream.next()
    print np.min(array[i:i+window_len]) - next[0], np.median(array[i:i+window_len]) - next[1], np.max(array[i:i+window_len])-next[2]
    i += 1

0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0
0 0.0 0


In [85]:
print get_tuple_stream_mean(min_median_max_stream(easy_stream(),1000), 3)

[4998069.499, 4998568.84, 4999068.18]


In [86]:
print get_tuple_stream_mean(min_median_max_stream(medium_stream(),1000), 3)

[0.02, 127.522, 254.98]


In [99]:
print get_tuple_stream_mean(min_median_max_stream(nightmare_stream(),1000), 3)

100%|██████████| 10000000/10000000 [21:47<00:00, 7645.54it/s]

[1000480.564, 499814213.467, 999008045.796]


In [92]:
print get_tuple_stream_mean(mean_var_stream(easy_stream(),1000), 2)

[4998568.84, 83386.216]


In [96]:
print get_tuple_stream_mean(mean_var_stream(medium_stream(),1000), 2)

100%|██████████| 10000000/10000000 [13:18<00:00, 12520.42it/s]

[127.494, 5452.516]


In [97]:
print get_tuple_stream_mean(mean_var_stream(nightmare_stream(),1000), 2)

100%|██████████| 10000000/10000000 [14:20<00:00, 11625.56it/s]

[499909716.256, 8.322315407932045e+16]


In [144]:
print get_tuple_stream_mean(mean_var_stream(nightmare_stream(),1000), 2)

[50012.148, 832704910.218]
